
# Imports

In [10]:
import pandas as pd
import numpy as np
import ast

In [ ]:
import string
!python -m spacy download en_core_web_sm -q
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS

2023-08-30 11:26:56.988615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 11:26:59.165197: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Data cleaning

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Final_project/top_1000_popular_movies_tmdb.csv", engine = "python")

In [3]:
df.head()

,Unnamed: 0,id,title,release_date,genres,original_language,vote_average,vote_count,popularity,overview,budget,production_companies,revenue,runtime,tagline
0,0,385687,Fast X,2023-05-17,"['Action', 'Crime', 'Thriller']",English,7.4,1347.0,8363.473,Over many missions and against impossible odds...,340000000.0,"['Universal Pictures', 'Original Film', 'One R...",6.520000e+08,142.0,The end of the road begins.
1,1,603692,John Wick: Chapter 4,2023-03-22,"['Action', 'Thriller', 'Crime']",English,7.9,2896.0,4210.313,"With the price on his head ever increasing, Jo...",90000000.0,"['Thunder Road', '87Eleven', 'Summit Entertain...",4.317692e+08,170.0,"No way back, one way out."
2,2,502356,The Super Mario Bros. Movie,2023-04-05,"['Animation', 'Family', 'Adventure', 'Fantasy'...",English,7.8,4628.0,3394.458,"While working underground to fix a water main,...",100000000.0,"['Universal Pictures', 'Illumination', 'Ninten...",1.308767e+09,92.0,NaN
3,3,569094,Spider-Man: Across the Spider-Verse,2023-05-31,"['Action', 'Adventure', 'Animation', 'Science ...",English,8.8,1160.0,2859.047,"After reuniting with Gwen Stacy, Brooklyn’s fu...",100000000.0,"['Columbia Pictures', 'Sony Pictures Animation...",3.135222e+08,140.0,It's how you wear the mask that matters
4,4,536437,Hypnotic,2023-05-11,"['Mystery', 'Thriller', 'Science Fiction']",English,6.5,154.0,2654.854,A detective becomes entangled in a mystery inv...,70000000.0,"['Studio 8', 'Solstice Productions', 'Ingeniou...",0.000000e+00,94.0,Control is an illusion.


In [4]:
df = df.drop(["Unnamed: 0", "id", "original_language", "vote_count", "popularity", "budget", "production_companies", "revenue"], axis = 1)

In [5]:
df[["tagline", "overview"]].isna().value_counts()

tagline  overview
False    False       7383
True     False       2541
         True          77
dtype: int64

In [6]:
no_overview = df[df["overview"].isna() == True]
len(no_overview)

77

In [7]:
df = df.dropna(subset = "overview")

In [8]:
df[["tagline", "overview"]].isna().value_counts()

tagline  overview
False    False       7383
True     False       2541
dtype: int64

In [20]:
df_embedding = df.copy(deep = True).reset_index(drop = True)
df_embedding["MovieID"] = df_embedding.index
df_embedding.insert(0, 'MovieID', df_embedding.pop('MovieID'))
df_embedding["genres"] = df_embedding["genres"].apply(lambda x : ast.literal_eval(x))
df_embedding["embeddind_prep"] = df_embedding["genres"].apply(lambda x:' '.join(x).strip()) + '. ' + df_embedding["tagline"].fillna('') + ' ' + df_embedding["overview"]

In [21]:
df_embedding

,MovieID,title,release_date,genres,vote_average,overview,runtime,tagline,embeddind_prep
0,0,Fast X,2023-05-17,"[Action, Crime, Thriller]",7.4,Over many missions and against impossible odds...,142.0,The end of the road begins.,Action Crime Thriller. The end of the road beg...
1,1,John Wick: Chapter 4,2023-03-22,"[Action, Thriller, Crime]",7.9,"With the price on his head ever increasing, Jo...",170.0,"No way back, one way out.","Action Thriller Crime. No way back, one way ou..."
2,2,The Super Mario Bros. Movie,2023-04-05,"[Animation, Family, Adventure, Fantasy, Comedy]",7.8,"While working underground to fix a water main,...",92.0,NaN,Animation Family Adventure Fantasy Comedy. Wh...
3,3,Spider-Man: Across the Spider-Verse,2023-05-31,"[Action, Adventure, Animation, Science Fiction]",8.8,"After reuniting with Gwen Stacy, Brooklyn’s fu...",140.0,It's how you wear the mask that matters,Action Adventure Animation Science Fiction. It...
4,4,Hypnotic,2023-05-11,"[Mystery, Thriller, Science Fiction]",6.5,A detective becomes entangled in a mystery inv...,94.0,Control is an illusion.,Mystery Thriller Science Fiction. Control is a...
...,...,...,...,...,...,...,...,...,...
9919,9919,Vivah,2006-11-10,"[Romance, Drama]",6.4,"Poonam, a traditionally brought-up young woman...",175.0,Marriage isn't a game...,Romance Drama. Marriage isn't a game... Poonam...
9920,9920,Dead Men Don't Wear Plaid,1982-05-21,"[Comedy, Mystery]",6.6,Juliet Forrest is convinced that the reported ...,89.0,Laugh...or I’ll blow your lips off!,Comedy Mystery. Laugh...or I’ll blow your lips...
9921,9921,Communion,1989-11-10,"[Drama, Horror, Science Fiction, Thriller]",5.3,A novelist's wife and son see him changed by a...,103.0,The true story of one man's terrifying journey...,Drama Horror Science Fiction Thriller. The tru...
9922,9922,Tess,1979-10-06,"[Drama, Romance]",7.0,A strong-willed peasant girl is sent by her fa...,186.0,She was born into a world where they called it...,Drama Romance. She was born into a world where...
